# Writing out a USGSCSM ISD from a PDS3 Mex HRSC image

In [1]:
import ale
from ale.drivers.mex_drivers import MexHrscPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
import json
import os

## Instantiating an ALE driver

ALE drivers are objects that define how to acquire common ISD keys from an input image format, in this case we are reading in a PDS3 image using NAIF SPICE kernels for exterior orientation data. If the driver utilizes NAIF SPICE kernels, it is implemented as a [context manager](https://docs.python.org/3/reference/datamodel.html#context-managers) and will furnish metakernels when entering the context (i.e. when entering the `with` block) and free the metakernels on exit. This maintains the integrity of spicelib's internal data structures. These driver objects are short-lived and are input to a formatter function that consumes the API to create a serializable file format. `ale.formatters` contains available formatter functions. 

The default config file is located at `ale/config.yml` and is copied into your home directory at `.ale/config.yml` on first use of the library. The config file can be modified using a text editor. `ale.config` is loaded into memory as a dictionary. It is used to find metakernels for different missions. For example, there is an entry for LRO that points to `/usgs/cpkgs/isis3/data/lro/kernels/mk/` by default. If you want to use your own metakernels, you will need to udpate this path. For example, if the metakernels are located in `/data/lrolrocnac/mk/` the LRO entry should be updated with this path. If you are using the default metakernels, then you do not need to update the path.

ALE has a two step process for writing out an ISD: 1. Instantiate your driver (in this case `LroLrocPds3LabelNaifSpiceDriver`) within a context and 2. pass the driver object into a formatter (in this case, `to_usgscsm`).  

Requirements:
 * A PDS3 LRO LROC NAC image
 * NAIF metakernels installed
 * Config file path for LRO (ale.config.lro) pointing to LRO NAIF metakernel directory 
 * A conda environment with ALE installed into it usisng the `conda install` command or created using the environment.yml file at the base of ALE.

In [2]:
# printing config displays the yaml formatted string
print(ale.config)

# config object is a dictionary so it has the same access patterns 
print('Mex spice directory:', ale.config['mex'])

# updating config for new LRO path in this notebook 
# Note: this will not change the path in `.ale/config.yml`. This change only lives in the notebook.
# ale.config['lro'] = '/data/lrolrocnac/mk/'

cassini: /usgs/cpkgs/isis3/data/cassini/kernels/mk/
dawn: /data/spice/dawn-m_a-spice-6-v1.0/dawnsp_1000/extras/mk
kaguya: /data/spice/SELENE/kernels/mk/
lro: /scratch/jlaura/spice/lro-l-spice-6-v1.0/lrosp_1000/extras/mk/
mdis: /work/users/sgstapleton/ale/
mex: /scratch/jlaura/spice/mex-e_m-spice-6-v1.0/mexsp_1000/EXTRAS/MK
mro: /data/spice/mro-m-spice-6-v1.0/mrosp_1000/extras/mk
spice_root: /data/spice/

Mex spice directory: /scratch/jlaura/spice/mex-e_m-spice-6-v1.0/mexsp_1000/EXTRAS/MK


In [3]:
# change to desired PDS3 image path 
fileName = '/work/users/sgstapleton/ale/h0010_0000_bl2.img'

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with MexHrscPds3NaifSpiceDriver(fileName) as driver:
    print(dir(driver))
    print(driver.sensor_frame_id)
    print(driver.ikid)
    print(driver.fikid)
    print(driver.line_exposure_duration)
    # pass driver instance into formatter function
    usgscsmString = to_usgscsm(driver)

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_file', '_label', '_metakernel', '_metakernel_dir', '_num_ephem', '_num_quaternions', 'center_ephemeris_time', 'crosstrack_summing', 'detector_center_line', 'detector_center_sample', 'detector_start_line', 'detector_start_sample', 'downtrack_summing', 'ephemeris_start_time', 'ephemeris_stop_time', 'ephemeris_time', 'exposure_duration', 'fikid', 'filter_number', 'focal2pixel_lines', 'focal2pixel_samples', 'focal_length', 'frame_chain', 'ikid', 'image_lines', 'image_samples', 'instrument_host_id', 'instrument_host_name', 'instrument_id', 'instrument_name', 'isis_naif_keywords

### Write ISD to disk 

ALE formatter functions generally return bytes or a string that can be written out to disk. ALE's USGSCSM formatter function returns a JSON encoded string that can be written out using any JSON library. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.


In [4]:
# load the json encoded string ISD
usgscsm_dict = json.loads(usgscsmString)

# strip the image file extension and append .json 
jsonFile = os.path.splitext(fileName)[0] + '.json'

# write to disk 
with open(jsonFile, 'w') as fp:
    json.dump(usgscsm_dict, fp)

In [5]:
import pvl 

fileName = '/work/users/sgstapleton/ale/h0010_0000_bl2.img'

label = pvl.load(fileName)
imagemeta = label['IMAGE']
label

PVLModule([
  ('PDS_VERSION_ID', 'PDS3')
  ('RECORD_TYPE', 'FIXED_LENGTH')
  ('RECORD_BYTES', 1348)
  ('FILE_RECORDS', 6868)
  ('LABEL_RECORDS', 7)
  ('^IMAGE_HEADER', 8)
  ('^IMAGE', 13)
  ('FILE_NAME', 'H0010_0000_BL2.IMG')
  ('DATA_SET_ID', 'MEX-M-HRSC-3-RDR-V3.0')
  ('DATA_SET_NAME', 'MARS EXPRESS HRSC RADIOMETRIC RDR V3.0')
  ('DETECTOR_ID', 'MEX_HRSC_BLUE')
  ('EVENT_TYPE', 'MARS-REGIONAL-STEREO-Vo-Fl-Pf')
  ('INSTRUMENT_HOST_ID', 'MEX')
  ('INSTRUMENT_HOST_NAME', 'MARS EXPRESS')
  ('INSTRUMENT_ID', 'HRSC')
  ('INSTRUMENT_NAME', 'HIGH RESOLUTION STEREO CAMERA')
  ('MISSION_NAME', 'MARS EXPRESS')
  ('MISSION_PHASE_NAME', 'MC_Phase_0')
  ('PROCESSING_LEVEL_ID', 2)
  ('PRODUCT_CREATION_TIME',
   datetime.datetime(2015, 3, 17, 9, 7, 21, tzinfo=<UTC>))
  ('PRODUCT_ID', 'H0010_0000_BL2.IMG')
  ('RELEASE_ID', 1)
  ('REVISION_ID', 0)
  ('SPACECRAFT_CLOCK_START_COUNT', '1/0021822627.31247')
  ('SPACECRAFT_CLOCK_STOP_COUNT', '1/0021823018.54464')
  ('START_TIME', datetime.datetime(2004, 1,

In [6]:
import struct

durations = []

with open(fileName, 'rb') as f:
#       ('RECORD_BYTES', 1348)
#   ('FILE_RECORDS', 6868)
    record_bytes = label['RECORD_BYTES']
    nrecords = label['FILE_RECORDS']
    imgstart_record = label['^IMAGE']
    imgstart = record_bytes * (imgstart_record - 1)
    nimg_recrods = nrecords - imgstart_record
    f.seek(imgstart)
    print(nimg_recrods)
    for record in range(nimg_recrods):
        rbytes = f.read(record_bytes)

        eph_time = struct.unpack('<d', rbytes[:8])[0]
        exposure_duration = struct.unpack('<f', rbytes[8:12])[0] / 1000   
        
        durations.append([record+1, eph_time,exposure_duration])
        
durations

6855


[[1, 127014784.67314196, 0.05461290740966797],
 [2, 127014784.72775486, 0.05461290740966797],
 [3, 127014784.78236777, 0.05461290740966797],
 [4, 127014784.83698067, 0.05461290740966797],
 [5, 127014784.89159358, 0.05461290740966797],
 [6, 127014784.94620648, 0.05461290740966797],
 [7, 127014785.00081939, 0.05461290740966797],
 [8, 127014785.05543229, 0.05461290740966797],
 [9, 127014785.1100452, 0.05461290740966797],
 [10, 127014785.16465892, 0.05461290740966797],
 [11, 127014785.21927182, 0.05461290740966797],
 [12, 127014785.27388473, 0.05461290740966797],
 [13, 127014785.32849763, 0.05461290740966797],
 [14, 127014785.38311054, 0.05461290740966797],
 [15, 127014785.43772344, 0.05461290740966797],
 [16, 127014785.49233635, 0.05461290740966797],
 [17, 127014785.54694925, 0.05461290740966797],
 [18, 127014785.6015634, 0.05461290740966797],
 [19, 127014785.6561763, 0.05461290740966797],
 [20, 127014785.7107892, 0.05461290740966797],
 [21, 127014785.76540211, 0.05461290740966797],
 [22,

In [7]:
print()